<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/05_xarray_landsat_era5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Xarray Landsat + ERA5 ET model run (TSEB)

---

Here we show how to run an ET model using input data from a GeoTiff image. The image was prepared as a Cloud-Optimized Geotiff (COG), so you can read it directly from the [url](https://raw.githubusercontent.com/kaust-halo/geeet/main/examples/data/landsat_era5_sample.tif), or you can download it. 

The data contains Landsat-derived data (`NDVI`, `albedo`, `radiometric_temperature`), as well as climate reanalysis (ERA5) data (`surface_pressure`, `air_temperature`, `wind_speed`, `solar_radiation`, and `thermal_radiation`). The data was exported for the same image (`LC09_172039_20230421`) and region shown at the end of notebook `03_eepredefined_landsat_era5.ipynb` (or you can use [this code editor script](https://code.earthengine.google.com/628832ffbd371d828a048e9de2f0903d)). The energy balance output variables (`Hc`, `Hs`, `LEc`, `LEs`, `G`, `Rns`, and `Rnc`) obtained using GEE were also exported for comparison. 

Here we run the model and compare the output from GEE and our local run using xarray. 

## Reading the data

Here we can read the data directly from the url, or you can download it and then replace the url with the local file.

In [ ]:
import rioxarray

url = "https://raw.githubusercontent.com/kaust-halo/geeet/main/examples/data/landsat_era5_sample.tif"
data = rioxarray.open_rasterio(url, band_as_variable=True)
data = (data.rename({band:data[band].attrs["long_name"] for band in data})
    .assign(
        LE = lambda x: x.LEc+x.LEs,
        H = lambda x: x.Hc+x.Hs,
        Rn = lambda x: x.Rnc+x.Rns 
    )
)
data

Let's keep only the inputs for our local model run:

In [ ]:
inputs = data[["NDVI", "albedo", "radiometric_temperature", 
"surface_pressure", "air_temperature", "wind_speed", "solar_radiation", "thermal_radiation"]]
inputs

The GEE ET model uses [ee.Image.pixelLonLat](https://developers.google.com/earth-engine/apidocs/ee-image-pixellonlat) internally whenever longitude and latitude information are needed. For the local ET model run, `lon` and `lat` need to be provided. Currently, there are two options. Either we can use `.rio.reproject("EPSG:4326")` to transform the `x` and `y` coordinates to lon/lat, and then provide these, or we can provide a single `lon`, `lat` coordinate. For this small region, let's choose the latter. 

We also need to specify other scalar information including: doy (day of year), time, Vza (viewing zenith angle), zU (wind speed measurement height), and zT (temperature measurement height). 

In [ ]:
scalar_inputs = dict(
    doy = 111,          # 2023-04-21; Day of year
    time = 10.98,       # Local observation time
    Vza = 0,            # Viewing zenith angle (degrees)
    longitude = 38.25,  # Longitude (degrees)
    latitude = 30.25,   # Latitude (degrees)
    zU = 10,            # Wind measurement height (m)
    zT = 2              # Temperature measurement height (m)
)

Finally, run the model:

In [ ]:
import geeet

xet = geeet.tseb.tseb_series(inputs, **scalar_inputs)
xet

## Comparison

First let's take an overview within the 0-350 W/m² range. 

In [ ]:
import xarray as xr
xdiff = xet - data

flux_name = "LE"
plot_kwargs = dict(cmap="viridis", vmin=0, vmax=350)

facets = (xr.concat([
    data[flux_name],
    xet[flux_name],
    xdiff[flux_name]
], dim="band")
.assign_coords({"band": ["GEE model", "xarray model", "difference"]})
.assign_attrs({"units": "W/m²"})
)
facetGrid = facets.plot(col="band", **plot_kwargs)
facetGrid.set_titles(template='{value}')

Further inspecting at a smaller LE scale, we can see the effect of using a single longitude/latitude:

In [ ]:
facets.sel(band="difference").plot(vmin=-0.1, vmax=0.1)

## Reprojecting

Let's try now using variable longitude and latitude. For this, we need to add them as variables to our dataset.

In [ ]:
import numpy as np

datab = data.rio.reproject("EPSG:4326")
xv, yv = np.meshgrid(datab.x, datab.y, indexing='xy')
lon, lat = np.meshgrid(datab.x, datab.y, indexing='xy')
datab = datab.assign(longitude=(["y","x"], lon), latitude=(["y","x"], lat))

inputsb = (datab[[
    "longitude", "latitude",
    "NDVI", "albedo", "radiometric_temperature", 
    "surface_pressure", "air_temperature", "wind_speed", 
    "solar_radiation", "thermal_radiation"]]
)

xetb = geeet.tseb.tseb_series(inputsb, **dict(
    doy = 111,          # 2023-04-21; Day of year
    time = 10.98,       # Local observation time
    Vza = 0,            # Viewing zenith angle (degrees)
    zU = 10,            # Wind measurement height (m)
    zT = 2              # Temperature measurement height (m)
))

xdiffb = xetb - datab # difference with the GEE model run

xdiffb.LE.plot(vmin=-0.1, vmax=0.1)

We can see that the differences are no longer longitude-dependent, and much smaller in magnitude. 